# Multiple dispatch

In this notebook we'll explore **multiple dispatch**, which is a key feature of Julia.

Multiple dispatch makes software *generic* and *fast*!

#### Starting with the familiar

To understand multiple dispatch in Julia, let's start with what we've already seen.

We can declare functions in Julia without giving Julia any information about the types of the input arguments that function will receive:

In [1]:
f(x) = x.^2

f (generic function with 1 method)

and then Julia will determine on its own which input argument types make sense and which do not:

In [2]:
f(10)

100

In [3]:
f([1, 2, 3])

3-element Vector{Int64}:
 1
 4
 9

#### Specifying the types of our input arguments

However, we also have the *option* to tell Julia explicitly what types our input arguments are allowed to have.

For example, let's write a function `foo` that only takes strings as inputs.

In [6]:
foo(x::String, y::String) = println("My inputs $x and $y are both strings!")

foo (generic function with 1 method)

We see here that in order to restrict the type of `x` and `y` to `String`s, we just follow the input argument name by a double colon and the keyword `String`.

Now we'll see that `foo` works on `String`s and doesn't work on other input argument types.

In [7]:
foo("hello", "hi!")

My inputs hello and hi! are both strings!


In [8]:
foo(3, 4)

LoadError: MethodError: no method matching foo(::Int64, ::Int64)

To get `foo` to work on integer (`Int`) inputs, let's tack `::Int` onto our input arguments when we declare `foo`.

In [9]:
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")

foo (generic function with 2 methods)

In [10]:
foo(3, 4)

My inputs x and y are both integers!


Now `foo` works on integers! But look, `foo` also still works when `x` and `y` are strings!

In [11]:
foo("hello", "hi!")

My inputs hello and hi! are both strings!


This is starting to get to the heart of multiple dispatch. When we declared

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
we didn't overwrite or replace
```julia
foo(x::String, y::String)
```
Instead, we just added an additional ***method*** to the ***generic function*** called `foo`.

A ***generic function*** is the abstract concept associated with a particular operation.

For example, the generic function `+` represents the concept of addition.

A ***method*** is a specific implementation of a generic function for *particular argument types*.

For example, `+` has methods that accept floating point numbers, integers, matrices, etc.

We can use the `methods` to see how many methods there are for `foo`.

In [12]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::String, y::String) in Main at In[6]:1
[2] foo(x::Int64, y::Int64) in Main at In[9]:1

Aside: how many methods do you think there are for addition?

In [13]:
methods(+)

# 206 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(x::T, y::T) where T<:Union{Float16, Float32, Float64} in Base at float.jl:383
[3] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:531
[4] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:537
[5] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:414
[8] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/Dates/src/periods.jl:378
[9] +(F::LinearAlgebra.Hessenberg, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/hessenberg.jl:641
[10] +(A::LinearAlgebra.UpperTriangular, B::LinearAlgebra.UpperTriangular) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/triangular.jl:638
[11] +(A::LinearAlgebra.UpperTriangular, B::LinearAlgebra.UnitUpperTriangular) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/triangular.jl:640
[12] +(x::LinearAlgebra.UpperTriangular, H::LinearAlgebra.UpperHessenberg) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/hessenberg.jl:109
[13] +(A::LinearAlgebra.UpperTriangular, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/special.jl:98
[14] +(A::LinearAlgebra.Symmetric, B::LinearAlgebra.Symmetric) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:469
[15] +(A::LinearAlgebra.Symmetric, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:486
[16] +(S::LinearAlgebra.Symmetric, D::LinearAlgebra.Diagonal) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/diagonal.jl:191
[17] +(A::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:15
[18] +(A::LinearAlgebra.Symmetric, B::SparseArrays.AbstractSparseMatrix) in SparseArrays at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:18
[19] +(A::LinearAlgebra.Symmetric{<:Real, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:25
[20] +(A::LinearAlgebra.Symmetric{var"#s885", S} where {var"#s885"<:Real, S<:(AbstractMatrix{<:var"#s885"})}, B::LinearAlgebra.Hermitian) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/symmetric.jl:484
[21] +(A::LinearAlgebra.Symmetric{<:Any, <:SparseArrays.AbstractSparseMatrix}, B::LinearAlgebra.Hermitian{<:Any, <:SparseArrays.AbstractSparseMatrix}) in SparseArrays at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/SparseArrays/src/linalg.jl:23
[22] +(J::LinearAlgebra.UniformScaling) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:149
[23] +(J::LinearAlgebra.UniformScaling, x::Number) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:144
[24] +(J1::LinearAlgebra.UniformScaling, J2::LinearAlgebra.UniformScaling) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:150
[25] +(J::LinearAlgebra.UniformScaling, B::BitMatrix) in LinearAlgebra at /home/simon/julia-1.8.5/share/julia/stdlib/v1.8/LinearAlgebra/src/uniformscaling.jl:152
[26] +(x::LinearAlgebra.UniformS

So, we now can call `foo` on integers or strings. When you call `foo` on a particular set of arguments, Julia will infer the types of the inputs and dispatch the appropriate method. *This* is multiple dispatch.

Multiple dispatch makes our code generic and fast. Our code can be generic and flexible because we can write code in terms of abstract operations such as addition and multiplication, rather than in terms of specific implementations. At the same time, our code runs quickly because Julia is able to call efficient methods for the relevant types.

To see which method is being dispatched when we call a generic function, we can use the @which macro:

In [14]:
@which foo(3, 4)

foo(x::Int64, y::Int64) in Main at In[9]:1

Let's see what happens when we use `@which` with the addition operator!

In [15]:
@which 3.0 + 3.0

+(x::T, y::T) where T<:Union{Float16, Float32, Float64} in Base at float.jl:383

And we can continue to add other methods to our generic function `foo`. Let's add one that takes the ***abstract type*** `Number`, which includes subtypes such as `Int`, `Float64`, and other objects you would think of as numbers:

In [16]:
foo(x::Number, y::Number) = println("My inputs x and y are both numbers!")

foo (generic function with 3 methods)

In [17]:
@which foo(3.0, 4.0)

foo(x::Number, y::Number) in Main at In[16]:1

This method for `foo` will work on, for example, floating point numbers:

In [18]:
foo(3.0, 4.0)

My inputs x and y are both numbers!


We can also add a fallback, duck-typed method for `foo` that takes inputs of any type:

In [19]:
foo(x, y) = println("I accept inputs of any type!")

foo (generic function with 4 methods)

Given the methods we've already written for `foo` so far, this method will be called whenever we pass non-numbers to `foo`:

In [20]:
v = rand(3)
foo(v, v)

I accept inputs of any type!


### Exercises

#### 9.1

Extend the function `foo`, adding a method that takes only one input argument, which is of type `Bool`, and returns "foo with one boolean!"

In [25]:
foo(x::Bool) = println("foo with one boolean!")

foo (generic function with 5 methods)

#### 9.2

Check that the method being dispatched when you execute 
```julia
foo(true)
```
is the one you wrote.

In [28]:
@which foo(true)

foo(x::Bool) in Main at In[25]:1

In [26]:
foo(true)

foo with one boolean!


In [27]:
@assert foo(true) == "foo with one boolean!"

foo with one boolean!


LoadError: AssertionError: foo(true) == "foo with one boolean!"